In [ ]:
import numpy as np

# 4x4 gridworld
n = 4
states = n * n
actions = ['up', 'down', 'left', 'right']
action_prob = 1 / 4.0     # equiprobable random policy

def step(state, action):
    """Return next_state and reward for taking action from a given state."""
    if state == 0 or state == 15:     # terminal states
        return state, 0

    row, col = divmod(state, n)

    if action == 'up':
        row = max(row - 1, 0)
    elif action == 'down':
        row = min(row + 1, n - 1)
    elif action == 'left':
        col = max(col - 1, 0)
    elif action == 'right':
        col = min(col + 1, n - 1)

    next_state = row * n + col
    return next_state, -1  # reward = -1 per move

# Initialize V(s)
V = np.zeros(states)
gamma = 1.0   # discount factor

# Perform ONE FULL iteration of policy evaluation
new_V = np.zeros(states)
for s in range(states):
    value = 0
    for a in actions:
        next_s, r = step(s, a)
        value += action_prob * (r + gamma * V[next_s])
    new_V[s] = value

print("Updated V(s) after ONE iteration of policy evaluation:\n")
print(new_V.reshape((4, 4)))


Updated V(s) after one full policy evaluation iteration:
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]


In [2]:
# q_learning_5x5_maze.py
import numpy as np
import random

n = 5
states = n * n
actions = [0,1,2,3]  # up, right, down, left

def step(s, a):
    i, j = divmod(s, n)
    if a == 0: i = max(i-1, 0)
    elif a == 1: j = min(j+1, n-1)
    elif a == 2: i = min(i+1, n-1)
    elif a == 3: j = max(j-1, 0)
    s2 = i*n + j
    reward = 1 if s2 == states-1 else -0.01
    done = (s2 == states-1)
    return s2, reward, done

def q_learning(episodes=200, alpha=0.1, gamma=0.99, eps=0.1):
    Q = np.zeros((states, 4))
    for _ in range(episodes):
        s = 0
        done = False
        while not done:
            if random.random() < eps:
                a = random.choice(actions)
            else:
                a = np.argmax(Q[s])
            s2, r, done = step(s, a)
            Q[s, a] += alpha * (r + gamma*np.max(Q[s2]) - Q[s,a])
            s = s2
    return Q

Q = q_learning()
print("Final Q-Table:")
print(Q)
print("\nOptimal Policy (0=up,1=right,2=down,3=left):")
print(np.argmax(Q, axis=1).reshape(5,5))


Final Q-Table:
[[ 1.56375892e-01  9.79476186e-02  8.63548906e-01  1.88386708e-01]
 [-1.28886006e-02 -1.31553884e-02  4.98921963e-01 -1.34957663e-02]
 [-8.78695040e-03 -8.58671622e-03 -7.97195011e-03 -9.44798204e-03]
 [-5.93210943e-03 -4.69564390e-03 -3.33453593e-03 -5.46702091e-03]
 [-3.89520190e-03 -2.99700100e-03 -3.82480310e-03 -4.01251730e-03]
 [ 2.95494252e-01  8.82781216e-01  2.06640542e-03  3.05764907e-01]
 [ 5.73695675e-02  7.91402915e-02  9.01929873e-01  3.30210653e-01]
 [-6.15398697e-03 -5.18376748e-03  6.25121052e-01 -5.48557204e-03]
 [-2.99700100e-03 -3.07719100e-03  3.53516806e-02 -2.99700100e-03]
 [-2.98720000e-03 -2.99700100e-03 -2.89810000e-03 -3.17502280e-03]
 [-9.48122924e-03  2.84821974e-01 -7.52532439e-03 -7.97205593e-03]
 [ 4.76709845e-01  9.21178811e-01  1.53508972e-01  2.40146721e-02]
 [ 2.15594097e-01  3.44167345e-01  9.40595156e-01  1.79095188e-01]
 [-2.09790100e-03 -1.99900000e-03  8.09565622e-01 -1.99900000e-03]
 [-1.99900000e-03 -1.99900000e-03 -1.90000000e-

In [ ]:
#Compare Greedy vs ε-Greedy (ε=0.1) on 7-armed Bandit (2000 steps)
# bandit_compare.py
import numpy as np

class Bandit:
    def __init__(self, k=7):
        self.k = k
        self.means = np.random.randn(k)

    def pull(self, arm):
        return self.means[arm] + np.random.randn()

def run_bandit(eps, steps=2000):
    bandit = Bandit(7)
    Q = np.zeros(7)
    N = np.zeros(7)
    rewards = []

    for _ in range(steps):
        if np.random.rand() < eps:
            a = np.random.randint(7)
        else:
            a = np.argmax(Q)
        r = bandit.pull(a)
        N[a] += 1
        Q[a] += (r - Q[a]) / N[a]
        rewards.append(r)

    return np.mean(rewards)

avg_greedy = run_bandit(eps=0.0)
avg_eps_greedy = run_bandit(eps=0.1)

print("Average Reward (Greedy):", avg_greedy)
print("Average Reward (ε-greedy, ε=0.1):", avg_eps_greedy)


Average Reward (Greedy): 1.3992878034256078
Average Reward (ε-greedy, ε=0.1): 0.6728783003284489


In [2]:
import numpy as np
import random

# 5x5 Maze (0 = free, 1 = wall)
maze = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 0, 1, 0]
])

n = 5
goal = (4, 4)

actions = [(0,1), (0,-1), (1,0), (-1,0)]  # right, left, down, up
num_actions = len(actions)

# Initialize Q-table
Q = np.zeros((n, n, num_actions))

alpha = 0.1    # learning rate
gamma = 0.95   # discount
epsilon = 0.1  # epsilon-greedy

def is_valid(x, y):
    return 0 <= x < n and 0 <= y < n and maze[x][y] == 0

def step(state, action_idx):
    x, y = state
    dx, dy = actions[action_idx]
    nx, ny = x + dx, y + dy

    if not is_valid(nx, ny):  # invalid move
        return state, -1

    if (nx, ny) == goal:
        return (nx, ny), 10  # reward for reaching goal

    return (nx, ny), -1  # step penalty

# Q-learning for 200 episodes
for episode in range(200):
    state = (0,0)  # start

    while state != goal:
        if random.uniform(0,1) < epsilon:
            action = random.randint(0, num_actions-1)  # explore
        else:
            action = np.argmax(Q[state])               # exploit

        next_state, reward = step(state, action)

        Q[state][action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][action])

        state = next_state

print("Final Q-table after 200 episodes:\n")
print(Q)


Final Q-table after 200 episodes:

[[[-3.74760923 -3.10274482  0.93744535 -3.22782046]
  [-2.965209   -3.13258001 -3.04368676 -3.12256937]
  [-1.72930324 -2.47325494 -2.41356137 -2.35559514]
  [ 0.20406742 -1.71863461 -1.63367063 -1.63367063]
  [-0.97112215 -1.01313019  2.80622194 -0.81704787]]

 [[-2.48131744 -2.30281944  2.04854602 -2.70189137]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [-0.40288278 -0.49502494  5.63245989 -0.38842822]]

 [[ 3.21225897 -1.34972175 -0.84803341 -1.81850096]
  [ 4.43489387 -0.6839706   0.27600488  0.54777983]
  [-0.50857206  0.21091386  5.72119759  1.48754788]
  [ 0.          0.          0.          0.        ]
  [ 0.49685683 -0.1995      8.20543317 -0.22771999]]

 [[ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 7.07499351  0.13223368 -0.52235411  1.50133259]
  [ 8.49999948  1.89866

In [3]:
import numpy as np
import random

maze = [
    [0,0,0,0,0],
    [0,1,1,1,0],
    [0,0,0,1,0],
    [1,1,0,0,0],
    [0,0,0,1,0]
]

n = 5
states = n * n
actions = [0,1,2,3]

def is_valid(i, j):
    return 0 <= i < n and 0 <= j < n and maze[i][j] == 0

def step(s, a):
    i, j = divmod(s, n)
    ni, nj = i, j

    if a == 0: ni = i - 1
    elif a == 1: nj = j + 1
    elif a == 2: ni = i + 1
    elif a == 3: nj = j - 1

    if not is_valid(ni, nj):
        return s, -0.05, False

    s2 = ni * n + nj
    goal = states - 1
    reward = 1 if s2 == goal else -0.01
    done = (s2 == goal)
    return s2, reward, done

def q_learning(episodes=200, alpha=0.1, gamma=0.99, eps=0.1):
    Q = np.zeros((states, 4))
    start = 0
    goal = states - 1

    if maze[n-1][n-1] == 1:
        raise ValueError("Goal cell is blocked.")

    for _ in range(episodes):
        s = start
        done = False
        while not done:
            if random.random() < eps:
                a = random.choice(actions)
            else:
                a = np.argmax(Q[s])
            s2, r, done = step(s, a)
            Q[s, a] += alpha * (r + gamma * np.max(Q[s2]) - Q[s, a])
            s = s2

    return Q

Q = q_learning()
print("Final Q-Table:")
print(Q)

print("\nOptimal Policy (0=up,1=right,2=down,3=left):")
print(np.argmax(Q, axis=1).reshape(n, n))


Final Q-Table:
[[-0.0041211  -0.02978522  0.86311609  0.03912375]
 [-0.02451804 -0.01605241 -0.03387308 -0.02337894]
 [-0.0282849   0.02056462 -0.02772844 -0.01879967]
 [-0.01051733  0.11224197 -0.01467131 -0.01269005]
 [-0.009995   -0.01698995  0.29978487 -0.00802535]
 [ 0.20691223  0.12742151  0.8826672   0.17406604]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-0.00365541 -0.005       0.59045606 -0.00951907]
 [ 0.37875687  0.90190419  0.06199827  0.13456533]
 [ 0.39162592  0.92117418  0.43531028  0.35101856]
 [ 0.52567302  0.34880392  0.94059477  0.42143799]
 [ 0.          0.          0.          0.        ]
 [-0.0020881  -0.005       0.90376364 -0.005     ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.31378572  0.96019957  0.04634931  0.40739171]
 [ 0.30743194  0.97999998  0.43899088  0.11752583]
 [ 0.39193752  0

In [ ]:
# ✅ 1. Monte Carlo Prediction (First-Visit MC)

import numpy as np, random

def mc_prediction(policy, env, episodes=5000, gamma=0.99):
    V = np.zeros(env.nS)
    returns = {s: [] for s in range(env.nS)}

    for _ in range(episodes):
        s = env.reset()
        trajectory = []
        done = False
        while not done:
            a = policy[s]
            s2, r, done, _ = env.step(a)
            trajectory.append((s, r))
            s = s2

        G = 0
        visited = set()
        for t in reversed(range(len(trajectory))):
            s, r = trajectory[t]
            G = r + gamma * G
            if s not in visited:
                visited.add(s)
                returns[s].append(G)
                V[s] = np.mean(returns[s])

    return V


In [ ]:
# 2. Monte Carlo Control (Exploring Starts — ES MC Control)
# Finds optimal Q* and greedy policy.

import numpy as np, random

def mc_control_es(env, episodes=5000, gamma=0.99):
    Q = np.zeros((env.nS, env.nA))
    policy = np.zeros(env.nS, dtype=int)
    returns = {(s,a): [] for s in range(env.nS) for a in range(env.nA)}

    for _ in range(episodes):
        s = random.randint(0, env.nS-1)
        a = random.randint(0, env.nA-1)
        trajectory = []

        s2, r, done, _ = env.step(a)
        trajectory.append((s,a,r))
        s = s2

        while not done:
            a = policy[s]
            s2, r, done, _ = env.step(a)
            trajectory.append((s,a,r))
            s = s2

        G = 0
        visited = set()
        for t in reversed(range(len(trajectory))):
            s,a,r = trajectory[t]
            G = r + gamma*G
            if (s,a) not in visited:
                visited.add((s,a))
                returns[(s,a)].append(G)
                Q[s,a] = np.mean(returns[(s,a)])
                policy[s] = np.argmax(Q[s])

    return Q, policy


In [ ]:
# 3. Monte Carlo Control (On-Policy ε-soft)

import numpy as np, random

def mc_control_eps(env, episodes=5000, eps=0.1, gamma=0.99):
    Q = np.zeros((env.nS, env.nA))
    returns = {(s,a): [] for s in range(env.nS) for a in range(env.nA)}
    policy = np.ones((env.nS, env.nA))*eps/env.nA

    for s in range(env.nS):
        policy[s,0] += 1-eps

    for _ in range(episodes):
        s = env.reset()
        trajectory = []
        done = False

        while not done:
            a = np.random.choice(env.nA, p=policy[s])
            s2, r, done, _ = env.step(a)
            trajectory.append((s,a,r))
            s = s2

        G = 0
        visited = set()
        for t in reversed(range(len(trajectory))):
            s,a,r = trajectory[t]
            G = r + gamma*G
            if (s,a) not in visited:
                visited.add((s,a))
                returns[(s,a)].append(G)
                Q[s,a] = np.mean(returns[(s,a)])

                best = np.argmax(Q[s])
                policy[s] = np.ones(env.nA)*eps/env.nA
                policy[s,best] += 1-eps

    return Q, policy


In [ ]:
# Perform monte carlo prediction over 3 states
# Monte Carlo Prediction (First Visit) on a 3-State MDP

import numpy as np

# We have 3 states: 0, 1, 2
states = [0, 1, 2]
gamma = 1.0
episodes = 5000

# Initialize V(s) = 0 for all states
V = np.zeros(3)

# For storing all returns of each state
returns = {0: [], 1: [], 2: []}

# ------------------------------------------------
# EPISODE GENERATION FUNCTION
# Deterministic transitions (example MDP):
# State 0 → State 1, reward = +1
# State 1 → State 2, reward = +2
# State 2 → Terminal, reward = 0
# ------------------------------------------------
def generate_episode():
    episode = []
    s = 0                   # start from state 0

    while True:
        if s == 0:
            s2, r = 1, 1    # state 0 gives reward 1 and moves to 1
        elif s == 1:
            s2, r = 2, 2    # state 1 gives reward 2 and moves to 2
        elif s == 2:
            return episode + [(2, 0)]   # state 2 is terminal with reward 0

        # store (state, reward)
        episode.append((s, r))
        s = s2              # move to next state


# ------------------------------------------------
# MONTE CARLO FIRST-VISIT PREDICTION
# ------------------------------------------------
for _ in range(episodes):

    ep = generate_episode()    # one complete episode
    G = 0                      # return

    visited = set()            # to track first-visit

    # loop backward through episode
    for s, r in reversed(ep):
        G = r + gamma * G      # compute return

        # Only first-visit update
        if s not in visited:
            visited.add(s)
            returns[s].append(G)
            V[s] = np.mean(returns[s])   # average return


# ------------------------------------------------
# PRINT RESULTS
# ------------------------------------------------
print("Monte Carlo Estimated State Values:")
print("V(0) =", V[0])
print("V(1) =", V[1])
print("V(2) =", V[2])


Monte Carlo Estimated State Values:
V(0) = 3.0
V(1) = 2.0
V(2) = 0.0
